In [1]:
# Run it once and Comment
# import sys
# sys.path.append('C:\\Shaukat\\code\\functions_implemented')
# print sys.path
# import functions_implemented as fi


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
import scipy.stats as stats
from datetime import datetime, timedelta

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from copy import deepcopy

from keras.models import load_model

%matplotlib inline





Using Theano backend.


In [3]:
# df_gfs = pd.read_csv("C://Shaukat//code//data_rep//gfs//csv_files//gfs_dunalley.csv")
df_bom = pd.read_csv("C://Shaukat//code//data_rep//bom//Woodbridge.csv")

In [4]:
# Setting up GFS
# df_gfs['Unnamed: 0'] = pd.to_datetime(df_gfs['Unnamed: 0'])
# df_gfs.set_index('Unnamed: 0', inplace = True)

# Setting up BOM
# df_bom['date_only'] = df_bom.UtcTime.dt.date
df_bom.UtcTime = pd.to_datetime(df_bom.UtcTime)
df_bom.set_index('UtcTime', inplace=True)

# Trim df_bom and df_gfs from november 2016
df_bom = df_bom.loc['2016-01-01 00:00:00':]
# df_gfs = df_gfs.loc['2016-11-01 00:00:00':]

In [5]:
df_bom['local_time_p1'] = df_bom['LocalTime'].shift(-1)

In [6]:
df_bom['local_time_p1'] = pd.to_datetime(df_bom['local_time_p1'])
df_bom['LocalTime'] = pd.to_datetime(df_bom['LocalTime'])

In [7]:
df_bom['local_time_diff'] = df_bom['local_time_p1'] - df_bom['LocalTime']


In [8]:
df_bom.head(5)
# df_bom.to_csv('C://Shaukat//code//data_rep//bom//EagleHawk_timecheck.csv')

,ObservationId,Timestamp,ParentLocationId,RelatedLocationId,ReadingId,LocalTime,TimeZone,Temperature,FeelsLike,DewPoint,...,WindDirection,WindDirectionCompass,WindSpeed,WindGust,RainSince9am,Pressure,RainfallLastHour,RainfallLast10Minutes,local_time_p1,local_time_diff
UtcTime,,,,,,,,,,,,,,,,,,,,,
2016-09-08 05:30:00,13616,1473276600,16,94255,8162,2016-09-08 05:30:00,Australia/Hobart,17.799999,17.799999,8.9,...,352,N,28,43,0.0,1014.0,0.0,0.0,2016-09-08 05:40:00,00:10:00
2016-09-08 05:40:00,13624,1473277200,16,94255,8179,2016-09-08 05:40:00,Australia/Hobart,17.000000,17.000000,8.9,...,3,N,19,22,0.0,0.0,0.0,0.0,2016-09-08 05:50:00,00:10:00
2016-09-08 05:50:00,13640,1473277800,16,94255,8196,2016-09-08 05:50:00,Australia/Hobart,16.700001,16.700001,9.4,...,10,N,15,15,0.0,0.0,0.0,0.0,2016-09-08 06:00:00,00:10:00
2016-09-08 06:00:00,13653,1473278400,16,94255,8213,2016-09-08 06:00:00,Australia/Hobart,16.700001,16.700001,9.6,...,10,N,17,22,0.0,1014.0,0.0,0.0,2016-09-08 06:10:00,00:10:00
2016-09-08 06:10:00,13668,1473279000,16,94255,8230,2016-09-08 06:10:00,Australia/Hobart,16.400000,16.400000,9.3,...,2,N,13,17,0.0,0.0,0.0,0.0,2016-09-08 06:40:00,00:30:00


In [ ]:
# df_bom.local_time_diff < timedelta(hours=1)

In [9]:
flag = True 
list_ind = []

for entry in range(0,len(df_bom.index)):
    if df_bom.local_time_diff[entry] > timedelta(hours=1):
        list_ind.append(entry)
        print 'gap between readings exceeds one hour\n'
        flag = False

print 'This must be True: ',flag

gap between readings exceeds one hour

gap between readings exceeds one hour

gap between readings exceeds one hour

gap between readings exceeds one hour

gap between readings exceeds one hour

gap between readings exceeds one hour

gap between readings exceeds one hour

gap between readings exceeds one hour

gap between readings exceeds one hour

gap between readings exceeds one hour

gap between readings exceeds one hour

This must be True:  False


In [10]:
list_ind

[66, 91, 460, 1739, 2590, 4858, 6337, 6783, 7195, 8446, 9162]

In [11]:
flag = True 
list_ind = []

for entry in range(0,len(df_bom.index)):
    if df_bom.local_time_diff[entry] > timedelta(days=1):
        list_ind.append(entry)
        print 'gap between readings exceeds one day\n'
        flag = False

print 'This must be True: ',flag

gap between readings exceeds one day

This must be True:  False


In [12]:
df_bom.local_time_diff[list_ind]

UtcTime
2016-09-09 05:00:00   2 days 18:10:00
Name: local_time_diff, dtype: timedelta64[ns]

In [13]:
list_ind

[91]